# Name matching
*Author: Pierre Adda*
## context:
We want to try models using  players statistics for each games, in order to predict that game's outcome. </br>
the players stats that we will use are the FIFA player's stats, found at [this url](https://www.kaggle.com/stefanoleone992/fifa-22-complete-player-dataset "Kaggle Fifa complete dataset").</br>
The games results with the players appearance can be found at [this url](https://data.world/dcereijo/player-scores).
This dataset includes 40k+ games from european leagues (including russia) with 20k+ players from 300+ clubs</br></br>
The main focus of this notebook is to link the players from the latter dataset to those in the FIFA dataset, in order to build our model training dataframe. </br>
This is not a trivial task as the names are either not always properly registered, not registered the same way, are not present from one dataset to another, or several players have the exact same name or a very similar name.

## Requirements

1. Python 3.8 or higher
2. Pandas and Numpy
3. the following libraries: unicode, tqdm (for loading bars), python-Levenshtein and fuzzywuzzy for "fuzzy" string matching:

<code>
!pip install unidecode -q  

!pip install python-Levenshtein -q  
!pip install fuzzywuzzy -q  
!pip install "tqdm>=4.9.0"
</code>
<br><br><br><br>

In [1]:
!pip install unidecode -q  

!pip install python-Levenshtein -q
!pip install fuzzywuzzy -q
!pip install "tqdm>=4.9.0"


## 1. Imports

In [2]:
import pandas as pd
import numpy as np
import unidecode
from fuzzywuzzy import fuzz #this amazing library uses levenshtein distance (string similarity scores) to match somewhat similar strings
from tqdm import tqdm
import re
tqdm.pandas()

/opt/conda/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
pd.options.display.max_columns =None

## 2. Functions

In [4]:
# This function preprocesses the FIFA datasets
# it removes diacritics, hyphens, apostrophes on player names and club. 
# "dj" is replaced to "d" to avoid recurrent ambiguity related to eastern european names

def decode_fifa(fifa_df):
    fifa_df.short_name = fifa_df.short_name.apply(unidecode.unidecode)
    fifa_df.long_name = fifa_df.long_name.apply(unidecode.unidecode)
    fifa_df.club_name = fifa_df.club_name.apply(unidecode.unidecode)

    fifa_df.short_name = fifa_df.short_name.str.lower().str.replace("'","").str.replace("-"," ").str.replace('dj','d')
    fifa_df.long_name = fifa_df.long_name.str.lower().str.replace("'","").str.replace("-"," ").str.replace('dj','d')
        
        
    return None

In [5]:
#This function returns a list of the best matching names index(es) and the the corresponding fuzz.token_set_ratio max value ( = string similarity max score)

def search_name(x,fifa_N):
    fuzz_scores = fifa_N.long_name.apply(lambda y: fuzz.token_set_ratio(x,y)).values   
    #each name in the "fifa" dataframe is assigned a similarity score with the selected name from the "players" dataset
    
    return np.argwhere(fuzz_scores == fuzz_scores.max()).squeeze() , fuzz_scores.max()

In [6]:
# This function uses the function search_name() on all names from the "players" dataframe 
# The function adds the best matching name(s) index(es) and fuzz score to the link dataframe
# it returns the number of names that might have matched correctly

def search_all_names(fifa_N):
    results = link.name.progress_apply(lambda x:search_name(x,fifa_N))
    
    link['fifa_index']= results.apply(lambda x : x[0])
    link['fuzz_score'] = results.apply(lambda x : x[1])
    
    return link.iloc[:10,:].loc[link.fuzz_score > 90,'fuzz_score'].shape

## 3. Data

### Game dataset
##### If you do not have the "game" dataset (players.csv, games.csv, appearance.csv, clubs.csv), please take the following steps:


1. create ``./data`` and ``./data/games/`` folders.  

2. execute the cell below

##### If you already have this dataset, make sure it is in the right folder and ignore the next cell

In [8]:
# Downloads all the dataset and store it in ./data/games/
appearance = pd.read_csv('https://query.data.world/s/xemgpklltd3hlau4swg2vafdctgacf')
clubs = pd.read_csv('https://query.data.world/s/bmpof22nmwcl7dc4s5kf5l2pjf6l62')
leagues = pd.read_csv('https://query.data.world/s/zmlqmpvqs4atuxn3rsdkdqv5wa6c5o')
games = pd.read_csv('https://query.data.world/s/ntedgrx2r6shpsvskopamknbnl7sfk')
players = pd.read_csv('https://query.data.world/s/jyeqrkxvhxmqxzqfac2s6kquuxrfuo')

appearance.to_csv("../data/games/appearance.csv")
clubs.to_csv("d../ata/games/clubs.csv")
leagues.to_csv("../data/games/leagues.csv")
games.to_csv("../data/games/games.csv")
players.to_csv("../data/games/players.csv")

In [13]:
appearance = pd.read_csv("../data/games/appearance.csv")
clubs = pd.read_csv("../data/games/clubs.csv")
leagues = pd.read_csv("../data/games/leagues.csv")
games = pd.read_csv("../data/games/games.csv")
players = pd.read_csv("../data/games/players.csv")


### Fifa Dataset
if you don't have this dataset, you can download it at [this url](https://www.kaggle.com/stefanoleone992/fifa-22-complete-player-dataset "fifa dataset").  
This dataset should be stored in `` ./data/FIFA/ ``

In [87]:
#Choose one of the FIFA dataset.
#The find_name function was initially designed to be used on all FIFA dataset at once, but trust me it's long enough using only one FIFA dataset

#Here we chose FIFA15.csv
fifa_df = pd.read_csv("../data/FIFA/players_15.csv")

## 4. Preprocessing

In [88]:
fifa_df = fifa_df.loc[:,['sofifa_id','short_name','long_name','club_team_id','club_name']]

# Some club_name values are NaN obect. Problem: None and Nan object cannot be treated as strings. 
# Thus, in order to preprocess all names, we will replace missing names with unmistakingly non existing names, i.e. "ZZZZZ"
fifa_df.loc[fifa_df.club_name.isnull(),'club_name']="ZZZZZ"

# Only now can we apply the function decode_fifa
decode_fifa(fifa_df)

In [71]:
players.name = players.name.str.replace("-"," ").str.replace('dj','d')

## 5. Name matching

In [100]:
#This dataframe will be used to link the players from the 'appearance' dataset
link=players.loc[:,['player_id','name']].copy()
link.head(1)

,player_id,name
0,38790,dmitri golubov


In [101]:
# Adds to each player the list of clubs in which that player has played at least one game, according to the "appearance" dataset

link['clubs'] = link.player_id.progress_apply(lambda x:
                    appearance.loc[appearance.player_id == x,'player_club_id'].unique() if type(x) == int
                                     else [appearance.loc[appearance.player_id == y,'player_club_id'].unique() for y in x]
                                    )

100%|██████████| 22432/22432 [00:26<00:00, 855.16it/s]


In [106]:
# Some players are not found in the appearance dataset. Since we use players that played games to train our models, we can get rid of those missing players.
print("number of players that haven't played a single game:",link.loc[ link.clubs.apply(len) == 0 ,:].shape[0])
link.drop(link.loc[ link.clubs.apply(len) == 0 ,:].index, inplace = True)
link.reset_index(drop = True,inplace = True)

number of players that haven't played a single game: 0


***The actual name searching cell :***

In [107]:
# this function will apply the fuzz.token_set_ratio function on (number of lines in fifa_df)*(number of lines in players df)
# for fifa15, this is ~300 million times. It is what it is... and it takes somewhat 1h30 to compute.
# So THIS IS A WARNING: it takes a LONG time.

search_all_names(fifa_df)


100%|██████████| 17639/17639 [1:41:52<00:00,  2.89it/s]


(7,)

In [109]:
# Let's save this dataframe that took so long to compute in the root folder
# To store this file somewhere else is left to the user's discretion

#don't forget to change the name if you used another fifa dataframe
link.to_csv("fifa15_res.csv")

## 6. Name filtering

**if you want to open a "link" dataframe that was previously saved, compile the following cell**

In [128]:
link = pd.read_csv('fifa15_res.csv', index_col=0)

# Reading a .csv tranforms lists into strings.
# now let's apply some preprocessing to transform a string of a list back into a list again 

import ast # ast is part of python standard's library. 
def transform_int_column(x):
    if x[0] == r'[':
        res = re.sub(r'(\d)\s+(\d)',r'\1 , \2',x.replace('\n',''))
        res = ast.literal_eval(res) 
        res = [int(y) for y in res]
    else:
        res = int(x)
    return res
link.fifa_index = link.fifa_index.apply(transform_int_column)

##### remove names with similarity score <90, which are unlikely to be correct

In [129]:
print(link.shape)
link = link.loc[link.fuzz_score > 90,:]
link.reset_index(drop = True,inplace = True)
print(link.shape)

(17639, 5)
(7309, 5)


##### for informative purpose, let's add the names corresponding to the fifa_index in a new column

In [130]:
# This is our way of accessing the names with multiple matches. 
link.loc[link.fifa_index.apply(type)!=int,:].head(1)

,player_id,name,clubs,fifa_index,fuzz_score
19,55472,rafael marques,['hellas-verona'],"[2004, 6868]",100


In [131]:
link.loc[link.fifa_index.apply(type)!=int,'name_compare'] = \
link.loc[link.fifa_index.apply(type)!=int,'fifa_index'].apply(lambda x : [fifa_df.long_name[y]for y in x])

link.loc[link.fifa_index.apply(type)==int,'name_compare'] = \
link.loc[link.fifa_index.apply(type)==int,'fifa_index'].apply(lambda x : fifa_df.long_name[x])

### for names with multiple matches: 
##### we can check if any clubs in which the player have played can be found in the clubs from the fifa dataset

We will compare the club names of the fifa 'candidates' to the list of clubs we can somehow link to the target player:</br>
from the appearance dataset, we have every player for every games with the club ID of those players. Therefore, we can have a partial player's club appartenance history, given that the player has played games...</br>
We can also add to that list the players current club found the player's dataset</br>
It is assumed that a club having multiple players with the very same name is very unlikely

In [134]:
fifa_df.head(1)

,sofifa_id,short_name,long_name,club_team_id,club_name
0,158023,l. messi,lionel andres messi cuccittini,241.0,FC Barcelona


In [135]:
link['clubs'] = link.player_id.progress_apply(lambda x:
                    appearance.loc[appearance.player_id == x,'player_club_id'].unique() if type(x) == int
                                     else [appearance.loc[appearance.player_id == y,'player_club_id'].unique() for y in x]
                                    )

#let's add the "current club" from the players dataset to the clubs obtained from appearance dataset, if that club is not already in that list:

link.clubs = link.progress_apply(lambda x: 
                                 np.append(x['clubs'],players.loc[players.player_id == x['player_id'],'current_club_id']) 
                                 if players.loc[players.player_id == x['player_id'],'current_club_id'].values[0] not in x['clubs']
                                 else x['clubs']
                                 ,axis = 1)

#Replace club_id with club names:

link.clubs = link.clubs.progress_apply(lambda x:
                                      [clubs.loc[clubs.club_id == y, 'name'] for y in x])
link.clubs = link.clubs.apply(lambda x: list(np.concatenate(x).flat))

# Now let's add the clubs corresponding to the FIFA indexes in a new column:

link['fifa_clubs'] = link.fifa_index.progress_apply(lambda x : 
                                         fifa_df.loc[x,'club_name']
                                        )

100%|██████████| 7309/7309 [00:00<00:00, 24266.83it/s]


In [136]:
#last cell created pandas series while we would have preferred numpy arrays... Let's fix this:
link.loc[link.fifa_clubs.apply(type) != str,'fifa_clubs'] = link.loc[link.fifa_clubs.apply(type) != str,'fifa_clubs'].apply(lambda x : x.values)
link.loc[link.fifa_clubs.apply(type) != str,:].head(1)

,player_id,name,clubs,fifa_index,fuzz_score,name_compare,fifa_clubs
19,55472,rafael marques,[hellas-verona],"[2004, 6868]",100,"[rafael marques pinto, rafael marques dias brito]","[Hellas Verona, US Creteil-Lusitanos]"


In [137]:
# There are some club_id that cannot be found in the "clubs" dataset
# Remove rows in "link" that have no clubs, and that have a similarity score inferior to 100 (max similarity) or multiple matches.
# Supposedly this should not be removing any rows, but we'll do it anyway. 

print(link.loc[(link.clubs.apply(len) == 0)&(link.name_compare.apply(type)!= str),:].shape)
print(link.loc[(link.clubs.apply(len) == 0)&(link.fuzz_score<100),:].shape)

link.drop(link.loc[(link.clubs.apply(len) == 0)&(link.name_compare.apply(type)!= str),:].index, inplace = True)
link.drop(link.loc[(link.clubs.apply(len) == 0)&(link.fuzz_score<100),:].index, inplace = True)

print(link.loc[(link.clubs.apply(len) == 0)&(link.name_compare.apply(type)!= str),:].shape)
print(link.loc[(link.clubs.apply(len) == 0)&(link.fuzz_score<100),:].shape)

(0, 7)
(0, 7)
(0, 7)
(0, 7)


<b> The next 2 cells' goal is to check the name similarity between 2 club lists.</br>
    - The first cell returns a list of the max string similarity score between all of fifa_index clubs and ANY of the clubs in which the "target" player has played.</br>
    - The second cell returns the position of the max string similarity score in that list, if this score is >90. This should correspond to the right player.

In [138]:
link['club_corr'] = None
link.loc[link.fifa_index.apply(type) != int,'club_corr'] = link.loc[link.fifa_index.apply(type) != int,:].progress_apply(lambda x:
                                                   [max([fuzz.token_set_ratio(z,y) for z in x['clubs']]) for y in x['fifa_clubs']]
                                                   ,axis = 1
                                                  )            

100%|██████████| 614/614 [00:00<00:00, 1590.16it/s]


In [139]:
link.loc[link.fifa_index.apply(type) != int,'club_corr'] = link.loc[link.fifa_index.apply(type) != int,:].club_corr.apply(lambda x : 
                     np.argmax(x) if max(x)>90 else None
                     )

#The column club_corr contains now the position of the right fifa_index when fifa_index is a list of pultiple indexes


<b> At this point, the names with multiple matches and no corresponding club names can be removed

In [140]:
link = link.loc[(link.club_corr.notnull()) | (link.fifa_index.apply(type) == int),:]

<b> For names with multiple matches, we found the right fifa_index.  
Let's keep it and get rid of the others in the columns fifa_index, fifa_club and name_compare

In [141]:
link.loc[~link.club_corr.isnull(),'fifa_index'] = \
link.loc[~link.club_corr.isnull(),:].apply(lambda x:
                                                         x['fifa_index'][int(x['club_corr'])]
                                                         ,axis = 1)

link.loc[~link.club_corr.isnull(),'name_compare'] = \
link.loc[~link.club_corr.isnull(),:].apply(lambda x:
                                                         x['name_compare'][int(x['club_corr'])]
                                                         ,axis = 1)

link.loc[~link.club_corr.isnull(),'fifa_clubs'] = \
link.loc[~link.club_corr.isnull(),:].apply(lambda x:
                                                         x['fifa_clubs'][int(x['club_corr'])]
                                                         ,axis = 1)

link.loc[~link.club_corr.isnull(),:].head(2)

,player_id,name,clubs,fifa_index,fuzz_score,name_compare,fifa_clubs,club_corr
19,55472,rafael marques,[hellas-verona],2004,100,rafael marques pinto,Hellas Verona,0.0
39,40028,martin pedersen,[hobro-ik],15460,100,martin pedersen,Hobro IK,1.0


**We checked club name correpondance on players with multiple matches, but for good measure we can also check this on other players with a fuzz score <95**

In [149]:
# get a list of fuzz score between fifa clubs and clubs from appearance and players datasets
mask = link.loc[link.club_corr.isnull(),:].index
link.loc[mask,'club_corr'] = link.progress_apply(lambda x:\
    [fuzz.token_set_ratio(x['fifa_clubs'],y) for y in x['clubs']]\
    ,axis = 1\
    )


100%|██████████| 6941/6941 [00:00<00:00, 11720.05it/s]


In [160]:
#keep the results for clubs with fuzz score > 80 (completely arbitrary)
mask = link.loc[link.club_corr.apply(type) == list,:].index #pretty much the same mask, obtained differently as link.club_corr.isnull() doesn't work anymore
link.loc[mask ,'club_corr'] = link.loc[mask ,'club_corr'].apply(lambda x:\
    np.argmax(x) if max(x) > 80 else x\
    )

In [208]:
# now we get rid of the players with a fuzz_score<95 and those with non-matching club name.
# for FIFA15 it amounts to ~60 players, so even if there might be some mistakes it's not very significant

link = link.loc[~((link.club_corr.apply(type)==list) & (link.fuzz_score<96) ),:]
link.drop(columns = ['club_corr','fuzz_score'], inplace = True)

In [209]:
link["sofifa_id"] = link.fifa_index.apply(lambda x: fifa_df.sofifa_id[x])
link.head(1)

,player_id,name,clubs,fifa_index,name_compare,fifa_clubs,sofifa_id
0,38790,dmitri golubov,[fk-ufa],5626,dmitriy golubov,FC Ufa,147752


In [210]:
#Now let's save it
link.to_csv('link_fifa15.csv')